In [34]:
import numpy as np
## packages
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import statsmodels as sm
%matplotlib inline

from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize, scale

In [35]:
#Generate some data to play with
np.random.seed(42)
pp = 6
n = 100
n_samples, n_features =n , pp
data_X = np.random.randn(n_samples, n_features)
coef = 3 * np.random.randn(n_features)
inds = np.arange(n_features)
np.random.shuffle(inds)
coef[inds[5:]] = 0  # sparsify coef
data_y = np.dot(data_X, coef)
# add noise
data_y += 0.02 * np.random.normal((n_samples,))

In [36]:
#Import packages
import numba
from numba import jit

In [37]:
residual = np.empty([n])
result = np.empty([n])
beta = np.random.randn(n_features)
k = 1
data = data_X
y = data_y
#Soft threshold operator
def S(z, gamma):
    if np.abs(z) - gamma > 0:
        return np.sign(z)*(np.abs(z) - gamma)
    else:
        return 0

In [27]:
@jit( cache=True)
def myUpdate(residual, result, beta, data, y, n,pp ,l1_ratio, alpha,z):
    for k in range(pp):
        #1.Calculate the vector of residuals for the partial fit 
        #print('for predictor',k, 'out of',pp)
        for j in range(n):
            #print('for each data point',j,'out of', n)
            #Calculate the fit excluding the k feature
            for i in range(pp):
                if i != k:
                    result[j] += beta[i]*data[j,i]
            residual[j] = y[j]-result[j]
            #print('obtain residual', residual[j], 'for', j)
        #2. Calculate the inner product  of each data point and its residual
        for j in range(n):
            #print('for each ', j, 'in', n)
            z+=(1/n)* data[j,k]*residual[j]
            #print('z', z)
        beta[k]=S(z, alpha*l1_ratio)/(1+alpha*(1-l1_ratio))
        #print('beta_k', beta[k])
    return beta

In [28]:
#1.Standardize the data#
X = scale(data, axis=0)
y =scale(y)

myUpdate(residual, result, beta, data, y,n,pp,1,.5,1)

array([ -5.25430510e+145,  -5.62060900e+145,  -3.21954706e+145,
         5.84449344e+145,   2.25591748e+146,  -1.84098983e+146])

In [29]:
for i in range(10):
    beta = myUpdate(residual, result, beta, data, y,n,pp,1,.5,1)
    print(beta)

[  1.70200391e+146   2.60080408e+146   4.35329714e+146   2.46255925e+146
  -1.00254040e+147  -4.56916399e+146]
[ -1.16994892e+146  -9.07274363e+146  -2.28143590e+147  -3.36667744e+147
   1.83234602e+147   4.74997745e+147]
[ -1.30829273e+147   1.41982424e+147   8.45041277e+147   2.14526772e+148
   1.38479768e+148  -8.75609193e+147]
[  7.88145624e+147   6.80813871e+147  -2.58663846e+148  -1.06287656e+149
  -1.92147098e+149  -1.60040451e+149]
[ -1.76047901e+148  -9.56301048e+148   8.66036323e+148   4.83812630e+149
   1.55823159e+150   2.34296023e+150]
[ -9.11625826e+148   8.34416288e+149  -4.29999877e+149  -2.27154635e+150
  -1.08095189e+151  -2.15653392e+151]
[  1.45834521e+150  -6.65119624e+150   2.58319051e+150   1.14982387e+151
   7.07185636e+151   1.67185383e+152]
[ -1.25058238e+151   5.04626666e+151  -1.37464775e+151  -6.00447490e+151
  -4.50342511e+152  -1.18773045e+153]
[  9.12104678e+151  -3.63721316e+152   5.42651042e+151   2.99378451e+152
   2.80607947e+153   8.00196373e+153]
[

array([ -1.16377794e+154,  -1.25320004e+154,   1.27661186e+152,
        -5.90174054e+154,  -5.24035321e+153,  -2.79614215e+154,
        -1.48987427e+154,   3.79304346e+154,   9.47534894e+153,
         3.77593284e+154,   4.46329102e+154,   6.12265027e+154,
         1.31134650e+153,  -4.27496494e+154,   1.80102178e+154,
        -4.06754933e+154,  -5.91891107e+153,  -9.61559676e+153,
         8.42276114e+154,   2.51779411e+154,   6.54618383e+154,
         8.73445102e+153,  -2.57998303e+153,   5.52645117e+151,
         1.54622789e+154,  -2.44008457e+154,   3.02430280e+154,
         5.13111267e+154,   7.78116973e+153,   1.00155715e+155,
         1.64495767e+154,   3.22578980e+154,  -1.81868635e+153,
         2.07919992e+154,   1.26734533e+155,   3.49950422e+154,
        -7.64518636e+154,  -5.47025769e+154,  -1.55413601e+154,
         3.22295883e+154,  -5.70818890e+153,   3.34054650e+154,
        -2.73367376e+154,  -1.41904375e+154,  -1.09867122e+153,
         4.94559377e+153,   1.74497985e+

-2.2721034670007657e+138

-1.2774197853472408e+138

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
        -0.23413696],
       [ 1.57921282,  0.76743473, -0.46947439,  0.54256004, -0.46341769,
        -0.46572975],
       [ 0.24196227, -1.91328024, -1.72491783, -0.56228753, -1.01283112,
         0.31424733],
       [-0.90802408, -1.4123037 ,  1.46564877, -0.2257763 ,  0.0675282 ,
        -1.42474819],
       [-0.54438272,  0.11092259, -1.15099358,  0.37569802, -0.60063869,
        -0.29169375],
       [-0.60170661,  1.85227818, -0.01349722, -1.05771093,  0.82254491,
        -1.22084365],
       [ 0.2088636 , -1.95967012, -1.32818605,  0.19686124,  0.73846658,
         0.17136828],
       [-0.11564828, -0.3011037 , -1.47852199, -0.71984421, -0.46063877,
         1.05712223],
       [ 0.34361829, -1.76304016,  0.32408397, -0.38508228, -0.676922  ,
         0.61167629],
       [ 1.03099952,  0.93128012, -0.83921752, -0.30921238,  0.33126343,
         0.97554513],
       [-0.47917424, -0.18565898, -1.10633497, -1.

991

In [ ]:
y